In [412]:
import IPython, ipywidgets, traitlets, requests, pandas, jinja2
from coffeetools import coffee
from bokeh.sampledata import iris
%reload_ext yamlmagic
%reload_ext autoreload
%autoreload 2

In [536]:
access_token = ipywidgets.Text('asdf')
access_token

In [530]:
class Projects( traitlets.HasTraits ):
    current = traitlets.List([])
    def __init__( self, *args, **kwargs ):
        self.catalog = {}   
        self.session = requests.Session()
        
    @traitlets.observe('current')
    def _update_projects(self, change, *args, **kwargs):
        for project_or_url in change['new']:
            self._github_api( project_or_url )
        self.df = self.to_dataframe()
                
    def _github_api( self, project_or_url, params = {} ):
        if not project_or_url.startswith('http'):
            map_to_project = True
            user, repo = project_or_url.split('/')
            url = None
        else:
            map_to_project = False
            url = project_or_url
        if not url:
            url = 'https://api.github.com/repos/' + user + '/' + repo
                
        request = requests.Request('GET', url=url, params=params.copy(), headers={'User-Agent': 'tonyfast'} )
        url = request.prepare().url
        if not url in self.catalog.keys():            
            request.params['access_token'] = globals()['access_token'].value
            self.catalog[url] = {
                'requests': self.session.send(request.prepare()),
                'time': str(pandas.datetime.now()),
                'params': params.copy(),
            } 
            if map_to_project:
                self.catalog[project_or_url] = self.catalog[url]
        return url
    
    def table( self ):
        return IPython.display.HTML("""<table id="catalog" class="table"></table> """)
        
    
    def interact( self ):
        columns = ipywidgets.SelectMultiple( options=self.df.columns.tolist() )
        columns.value = ('url',)
        return IPython.display.display(columns, ipywidgets.interact( lambda i = [0, self.df.shape[0]-10 ]: self.update_table( i, list(columns.value) ))),
    
    def update_table( self, i, columns ):
        return IPython.display.Javascript(coffee.compile(jinja2.Template("""
        data = {{df}}
        table = d3.select '#catalog'
        header = table.selectAll 'tr.header'
            .data [ '', data['columns'] ]
        header.enter().append('tr').classed 'header', true
        header.each (d)->
            cells = d3.select @
                .selectAll 'th'
                .data d
            cells.enter().append 'th'
            cells.text (d)-> d
            cells.exit().remove()
        header.exit().remove()
        value = table.selectAll 'tr.value'
            .data data['data']
        value.enter().append('tr').classed 'value', true
        value.each (d)->
            cells = d3.select @
                .selectAll 'td'
                .data d
            cells.enter().append 'td'
            cells.text (d)-> d
            cells.exit().remove()  
        value.exit().remove()
        """).render(df = self.df[columns].iloc[i:i+10].to_json(orient='split'))))
    
    def more( self, url, params = {}, paginate=False ):
        if paginate:
            last_page = [v['params']['page'] for k, v in self.catalog.items() if k.startswith(url) and 'page' in v['params']]
            params['page'] = max(last_page) + 1 if last_page else 1
        url = self._github_api( url, params = params )
        self.df = self.to_dataframe()
        return self.catalog[url]
    
    def update( self, project_name ):
        tmp = list( self.current )
        tmp.append( project_name )
        self.current = tmp
        
    def to_dataframe( self ):
        v = []
        for _v in self.catalog.values():
            _v = _v['requests'].json()
            if isinstance( _v, list ):
                v.extend(_v)
        return pandas.DataFrame(v)
projects = Projects()

In [531]:
projects.current = ['jupyter/notebook', 'bokeh/bokeh']

In [532]:
iss = projects.more('https://api.github.com/repos/jupyter/notebook/issues', paginate=True)

In [533]:
iss = projects.more('https://api.github.com/repos/jupyter/notebook/issues', paginate=True)

In [534]:
projects.interact()

<IPython.core.display.Javascript object>

In [535]:
projects.table()

In [402]:
def click( button, **kwargs ):
    projects.more( button.value, paginate=True)

In [403]:
ipywidgets.SelectMultiple( options = projects.current, width = 800 )

In [302]:
buttons = []
for key, value in project.items():
    if key in ['issues_url','events_url']:
        if '{' in value:
            value = value.split('{')[0]
        button = ipywidgets.Button( description = key.split('_url')[0] )
        button.value = value 
        button.on_click( click )
        buttons.append( button )

In [303]:
ipywidgets.Box(children=buttons)